In [85]:
import torch

In [97]:
train_model = './trained_model/mobilenet_fpn7_512_bdd100k+MOT17_E200_0313.pth'
train_model_dict = torch.load(train_model, map_location='cpu')

In [98]:
train_model_dict = train_model_dict['model']

In [88]:
print(train_model_dict)

OrderedDict([('backbone.features.0.0.weight', tensor([[[[ 2.51458e-03, -8.22596e-04,  2.82740e-03],
          [ 6.24749e-03, -4.83819e-03,  1.30760e-03],
          [ 2.01055e-03, -7.11787e-03, -2.80779e-03]],

         [[ 1.52323e-03, -1.12695e-03,  2.87461e-03],
          [ 3.81313e-03, -6.26243e-03, -3.97644e-04],
          [ 2.16364e-03, -6.28027e-03, -1.50006e-03]],

         [[-4.80983e-03, -3.84517e-03, -1.89897e-03],
          [-2.13620e-03, -5.58025e-03, -2.90187e-03],
          [-5.05745e-03, -6.37340e-03, -4.61546e-03]]],


        [[[-5.96940e-03,  4.60231e-02, -6.54168e-03],
          [ 8.63875e-02,  1.75362e-01,  4.66888e-02],
          [-6.98407e-02, -1.63483e-01, -4.00627e-02]],

         [[ 5.07332e-03,  4.06186e-02,  1.56541e-02],
          [ 8.57047e-02,  1.85617e-01,  3.19123e-02],
          [-8.49343e-02, -1.96208e-01, -5.82290e-02]],

         [[-4.88100e-03,  2.00267e-02, -4.28854e-03],
          [ 5.83426e-02,  1.28355e-01,  3.19843e-02],
          [-5.91504e-02,

In [89]:
from models_ssd import build_ssd_model
from ssd.config import cfg

ssd_cfg_path = './mobilenet_fpn7.yaml'
cfg.merge_from_file(ssd_cfg_path)
cfg.freeze()

model = build_ssd_model(cfg, 14455)

-------build_backbone---------- mobilenet_v2_fpn7
-------build_backbone.PRETRAINED---------- True
-------build_box_head---------- SSDBoxHead
-------make_box_predictor---------- SSDLiteBoxPredictor


In [90]:
need_training_model_dict = model.state_dict()

In [91]:
for layer_name in train_model_dict.keys():
    need_training_model_dict[layer_name] = train_model_dict[layer_name]

In [92]:

model.load_state_dict(need_training_model_dict)

<All keys matched successfully>

In [93]:
model_info(model)


layer                                               name  gradient   parameters                shape           mu        sigma
    0                       backbone.features.0.0.weight      True          864        [32, 3, 3, 3]      -0.0012       0.0966
    1                       backbone.features.0.1.weight      True           32                 [32]       0.0797       0.0564
    2                         backbone.features.0.1.bias      True           32                 [32]        0.105        0.134
    3                backbone.features.1.conv.0.0.weight      True          288        [32, 1, 3, 3]      0.00417        0.177
    4                backbone.features.1.conv.0.1.weight      True           32                 [32]        0.114       0.0818
    5                  backbone.features.1.conv.0.1.bias      True           32                 [32]        0.209        0.195
    6                  backbone.features.1.conv.1.weight      True          512       [16, 32, 1, 1]       0.0

In [94]:
for i, (name, p) in enumerate(model.named_parameters()):
    if i < 300:  # if layer < 75
        p.requires_grad = False 

In [95]:
model_info(model)


layer                                               name  gradient   parameters                shape           mu        sigma
    0                       backbone.features.0.0.weight     False          864        [32, 3, 3, 3]      -0.0012       0.0966
    1                       backbone.features.0.1.weight     False           32                 [32]       0.0797       0.0564
    2                         backbone.features.0.1.bias     False           32                 [32]        0.105        0.134
    3                backbone.features.1.conv.0.0.weight     False          288        [32, 1, 3, 3]      0.00417        0.177
    4                backbone.features.1.conv.0.1.weight     False           32                 [32]        0.114       0.0818
    5                  backbone.features.1.conv.0.1.bias     False           32                 [32]        0.209        0.195
    6                  backbone.features.1.conv.1.weight     False          512       [16, 32, 1, 1]       0.0

In [ ]:
for name, p in model.named_parameters():
    if 'id' not in name:
        p.requires_grad = False